In [229]:
import numpy as np
import pandas as pd

In [230]:

import pyspark
from pyspark.sql import SparkSession

In [231]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

In [232]:
QuestionResponseFact = spark.read.csv('QuestionResponseFact.csv', header=True,inferSchema=None)
QuestionDimension = spark.read.csv('QuestionDimension.csv', header=True,inferSchema=None)
CourseDimension = spark.read.csv('CourseDimension.csv', header=True,inferSchema=None)
CourseGradeFact = spark.read.csv('CourseGradeFact.csv', header=True,inferSchema=None)
EnrollmentFact = spark.read.csv('EnrollmentFact.csv', header=True,inferSchema=None)
ResponseDimension = spark.read.csv('ResponseDimension.csv', header=True,inferSchema=None)
StudentFact = spark.read.csv('StudentFact.csv', header=True,inferSchema=None)
QuestionResponseFact.createOrReplaceTempView("QuestionResponseFact")
QuestionDimension.createOrReplaceTempView("QuestionDimension")
CourseDimension.createOrReplaceTempView("CourseDimension")
CourseGradeFact.createOrReplaceTempView("CourseGradeFact")
EnrollmentFact.createOrReplaceTempView("EnrollmentFact")
ResponseDimension.createOrReplaceTempView("ResponseDimension")
StudentFact.createOrReplaceTempView("StudentFact")


In [233]:
spark.sql("SELECT * FROM QuestionResponseFact").show()
spark.sql("SELECT * FROM QuestionDimension").show()
spark.sql("SELECT * FROM CourseDimension").show()
spark.sql("SELECT * FROM CourseGradeFact").show()
spark.sql("SELECT * FROM EnrollmentFact").show()
spark.sql("SELECT * FROM ResponseDimension").show()
spark.sql("SELECT * FROM StudentFact").show()


+------------------+---------+----------+----------+
|questionresponseid|studentid|questionid|responseid|
+------------------+---------+----------+----------+
|           9990128|        8|         2|         1|
|           9990148|        8|         4|         1|
|           9990149|        9|         4|         1|
|           9990224|        4|         2|         2|
|           9990229|        9|         2|         2|
|           9990233|        3|         3|         2|
|           9990234|        4|         3|         2|
|           9990235|        5|         3|         2|
|           9990236|        6|         3|         2|
|           9990239|        9|         3|         2|
|           9990314|        4|         1|         3|
|           9990321|        1|         2|         3|
|           9990322|        2|         2|         3|
|           9990323|        3|         2|         3|
|           9990325|        5|         2|         3|
|           9990332|        2|         3|     

In [234]:
StudentFactXEnrollmentFactXCourseGradeFact = spark.sql("SELECT StudentFact.studentid, \
            StudentFact.degree, \
          StudentFact.campus, StudentFact.sex, StudentFact.ethnicity, StudentFact.age, StudentFact.race, \
          CourseGradeFact.courseid, CourseGradeFact.coursegrade \
          FROM StudentFact \
          INNER JOIN \
          EnrollmentFact ON StudentFact.studentid=EnrollmentFact.studentid \
          INNER JOIN \
          CourseGradeFact ON StudentFact.studentid=CourseGradeFact.studentid")
StudentFactXEnrollmentFactXCourseGradeFact.show()

+---------+-------------+--------+---+---------------+---+-----+--------+-----------+
|studentid|       degree|  campus|sex|      ethnicity|age| race|courseid|coursegrade|
+---------+-------------+--------+---+---------------+---+-----+--------+-----------+
|        1|       Master|   Texas|  F|           null| 24|White|       4|          3|
|        1|       Master|   Texas|  F|           null| 24|White|       3|          2|
|        1|       Master|   Texas|  F|           null| 24|White|       3|          3|
|        1|       Master|   Texas|  F|           null| 24|White|       3|          4|
|        1|       Master|   Texas|  F|           null| 24|White|       2|2.799999952|
|        1|       Master|   Texas|  F|           null| 24|White|       1|          3|
|        2|       Master|   Texas|  F|           null| 23|White|       4|          2|
|        2|       Master|   Texas|  F|           null| 23|White|       3|1.299999952|
|        2|       Master|   Texas|  F|           null|

In [235]:
from pyspark.sql.functions import *

## BICOP

In [236]:
StudentFactXEnrollmentFactXCourseGradeFact.withColumn("BICOP",
            when((StudentFactXEnrollmentFactXCourseGradeFact['race']=='Black') | (StudentFactXEnrollmentFactXCourseGradeFact['race']=='Alaska Native') | 
           (StudentFactXEnrollmentFactXCourseGradeFact['race']=='Native Hawaiian') | (StudentFactXEnrollmentFactXCourseGradeFact['race']=='Native Hawaiian ') | 
           (StudentFactXEnrollmentFactXCourseGradeFact['race']=='Pacific Islander') | (StudentFactXEnrollmentFactXCourseGradeFact['race']=='American Indian') | 
           (StudentFactXEnrollmentFactXCourseGradeFact['race']=='Asian') |
                 (StudentFactXEnrollmentFactXCourseGradeFact.ethnicity == 'Hispanic/Latinx'), 
1).otherwise(0)).show()

+---------+-------------+--------+---+---------------+---+-----+--------+-----------+-----+
|studentid|       degree|  campus|sex|      ethnicity|age| race|courseid|coursegrade|BICOP|
+---------+-------------+--------+---+---------------+---+-----+--------+-----------+-----+
|        1|       Master|   Texas|  F|           null| 24|White|       4|          3|    0|
|        1|       Master|   Texas|  F|           null| 24|White|       3|          2|    0|
|        1|       Master|   Texas|  F|           null| 24|White|       3|          3|    0|
|        1|       Master|   Texas|  F|           null| 24|White|       3|          4|    0|
|        1|       Master|   Texas|  F|           null| 24|White|       2|2.799999952|    0|
|        1|       Master|   Texas|  F|           null| 24|White|       1|          3|    0|
|        2|       Master|   Texas|  F|           null| 23|White|       4|          2|    0|
|        2|       Master|   Texas|  F|           null| 23|White|       3|1.29999

In [237]:
StudentFactXEnrollmentFactXCourseGradeFact.show()

+---------+-------------+--------+---+---------------+---+-----+--------+-----------+
|studentid|       degree|  campus|sex|      ethnicity|age| race|courseid|coursegrade|
+---------+-------------+--------+---+---------------+---+-----+--------+-----------+
|        1|       Master|   Texas|  F|           null| 24|White|       4|          3|
|        1|       Master|   Texas|  F|           null| 24|White|       3|          2|
|        1|       Master|   Texas|  F|           null| 24|White|       3|          3|
|        1|       Master|   Texas|  F|           null| 24|White|       3|          4|
|        1|       Master|   Texas|  F|           null| 24|White|       2|2.799999952|
|        1|       Master|   Texas|  F|           null| 24|White|       1|          3|
|        2|       Master|   Texas|  F|           null| 23|White|       4|          2|
|        2|       Master|   Texas|  F|           null| 23|White|       3|1.299999952|
|        2|       Master|   Texas|  F|           null|

In [238]:
QuestionResponseFactXQuestionDimension = spark.sql('select QuestionResponseFact.studentid, \
        QuestionDimension.questionid, QuestionDimension.question \
        from QuestionResponseFact \
        inner join QuestionDimension \
        on QuestionResponseFact.questionid = QuestionDimension.questionid')
QuestionResponseFactXQuestionDimension.show()

+---------+----------+--------------------+
|studentid|questionid|            question|
+---------+----------+--------------------+
|        8|         2|I am satisfied wi...|
|        8|         4|I am satisfied wi...|
|        9|         4|I am satisfied wi...|
|        4|         2|I am satisfied wi...|
|        9|         2|I am satisfied wi...|
|        3|         3|I am satisfied wi...|
|        4|         3|I am satisfied wi...|
|        5|         3|I am satisfied wi...|
|        6|         3|I am satisfied wi...|
|        9|         3|I am satisfied wi...|
|        4|         1|I am satisfied wi...|
|        1|         2|I am satisfied wi...|
|        2|         2|I am satisfied wi...|
|        3|         2|I am satisfied wi...|
|        5|         2|I am satisfied wi...|
|        2|         3|I am satisfied wi...|
|        1|         4|I am satisfied wi...|
|        2|         4|I am satisfied wi...|
|        3|         4|I am satisfied wi...|
|        7|         4|I am satis

## Second 2 last bullet point

In [239]:
QuestionResponseFact

DataFrame[questionresponseid: string, studentid: string, questionid: string, responseid: string]

In [240]:
QuestionResponseFactXresponse = spark.sql('select QuestionResponseFact.studentid, \
        QuestionResponseFact.questionid, \
        ResponseDimension.response, ResponseDimension.responseid \
        from QuestionResponseFact \
        inner join ResponseDimension \
        on QuestionResponseFact.responseid = ResponseDimension.responseid')
QuestionResponseFactXresponse.show()

+---------+----------+--------------------+----------+
|studentid|questionid|            response|responseid|
+---------+----------+--------------------+----------+
|        8|         2|(1) Strongly disa...|         1|
|        8|         4|(1) Strongly disa...|         1|
|        9|         4|(1) Strongly disa...|         1|
|        4|         2|        (2) Disagree|         2|
|        9|         2|        (2) Disagree|         2|
|        3|         3|        (2) Disagree|         2|
|        4|         3|        (2) Disagree|         2|
|        5|         3|        (2) Disagree|         2|
|        6|         3|        (2) Disagree|         2|
|        9|         3|        (2) Disagree|         2|
|        4|         1|         (3) Neutral|         3|
|        1|         2|         (3) Neutral|         3|
|        2|         2|         (3) Neutral|         3|
|        3|         2|         (3) Neutral|         3|
|        5|         2|         (3) Neutral|         3|
|        2

In [241]:
from pyspark.sql.types import IntegerType
QuestionResponseFactXresponse = QuestionResponseFactXresponse.withColumn("responseid", QuestionResponseFactXresponse["responseid"].cast(IntegerType()))


### How satisfied are students with our program per survey question (i.e. instruction, operations, tuition, and communication)

In [242]:
QuestionResponseFactXresponse.groupby("questionid").avg("responseid").show()

+----------+------------------+
|questionid|   avg(responseid)|
+----------+------------------+
|         3|3.2036727879799667|
|         1|3.9382303839732886|
|         4|3.5692821368948247|
|         2|3.1803005008347247|
+----------+------------------+



##   

## Also BICOP Count

### Number of white Hispanic/Latinx

In [243]:
StudentFact[(StudentFact['ethnicity']=='Hispanic/Latinx') & (StudentFact['race']=='White')].count()

29

### Number of all Hispanic/Latinx regardless of race

In [244]:
StudentFact[StudentFact['ethnicity']=='Hispanic/Latinx'].count()

55

### Number of non-white Hispanic/Latinx

In [245]:
55-29 

26

## Note: In the dataset there are two Native Hawaiian . . .
##            one with a space after and one regular

In [246]:
StudentFact[(StudentFact['race']=='Black') | (StudentFact['race']=='Alaska Native') | 
           (StudentFact['race']=='Native Hawaiian') | (StudentFact['race']=='Native Hawaiian ') | 
           (StudentFact['race']=='Pacific Islander') | (StudentFact['race']=='American Indian') | 
           (StudentFact['race']=='Asian') ].count()

360

### Number of BIPOC's with race white but ethnicity Hispanic/Latinx

In [247]:
360+29

389

## Last bullet point

In [248]:
spark.sql('SELECT studentid, courseid, MAX(coursegrade) FROM CourseGradeFact GROUP BY studentid, courseid;').show()

+---------+--------+----------------+
|studentid|courseid|max(coursegrade)|
+---------+--------+----------------+
|        1|       1|               3|
|        1|       2|     2.799999952|
|        1|       3|               4|
|        1|       4|               3|
|       10|       1|     3.200000048|
|       10|       2|     3.599999905|
|       10|       3|     3.099999905|
|       10|       4|     2.099999905|
|      100|       1|     3.700000048|
|      100|       2|     3.700000048|
|      100|       3|     3.200000048|
|      100|       4|     3.400000095|
|      101|       1|     2.900000095|
|      101|       2|     3.200000048|
|      101|       3|     2.700000048|
|      101|       4|               4|
|      102|       1|     2.299999952|
|      102|       2|     2.900000095|
|      102|       3|     2.700000048|
|      102|       4|               4|
+---------+--------+----------------+
only showing top 20 rows



## 1

### Avegrage


In [292]:
avg1 = QuestionResponseFactXresponse.groupby("questionid").avg("responseid")

In [293]:
avg1.toPandas()

,questionid,avg(responseid)
0,3,3.203673
1,1,3.938230
2,4,3.569282
3,2,3.180301


### Sum

In [272]:
sum1 = QuestionResponseFactXresponse.groupby("questionid").sum("responseid")

In [273]:
sum1.toPandas()  

,questionid,sum(responseid)
0,3,1919
1,1,2359
2,4,2138
3,2,1905
